## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

In [9]:
import mysql.connector

config = {
  'user': 'root',
  'password': 'root',
  'host': 'localhost',
  'unix_socket': '/Applications/MAMP/tmp/mysql/mysql.sock',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)
print(cnx)


Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Essse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python

In [10]:
cursor = cnx.cursor()

## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [13]:
try:
    cursor.execute("CREATE DATABASE IF NOT EXISTS dbprodutos;")
except mysql.connector.Error as err:
    print("Failed creating database: {}".format(err))

Failed creating database: 1007: Can't create database 'dbprodutos'; database exists


In [14]:
cursor.execute("SHOW DATABASES;")

for db in cursor:
    print(db)

('information_schema',)
('CLUBE_DO_LIVRO',)
('dbprodutos',)
('mysql',)
('performance_schema',)
('sys',)


## Criando uma tabela

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [15]:
import pandas as pd
df_livros = pd.read_csv("../data/tabela_livros.csv")
df_livros.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,65cce385ddadb93e4611d6c2,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,65cce385ddadb93e4611d6fb,Modelagem preditiva,livros,70.49,1.337820,2023-02-01,Isabella Pereira,PE,4,cupom,1,-8.38,-37.86
2,65cce385ddadb93e4611d715,Ciência de dados com python,livros,50.51,10.245086,2022-03-01,Thiago Silva,MG,1,cartao_credito,6,-18.10,-44.38
3,65cce385ddadb93e4611d719,Modelagem preditiva,livros,71.64,1.399054,2023-03-01,Mariana Ferreira,SP,4,cartao_credito,2,-22.19,-48.79
4,65cce385ddadb93e4611d71b,Dashboards com Power BI,livros,43.35,3.248105,2023-03-01,Rafael Costa,BA,5,cartao_credito,2,-13.29,-41.71


In [16]:
df_livros.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [17]:
df_livros.shape

(2226, 13)

In [18]:
try:
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS dbprodutos.tb_livros(
            id VARCHAR(100),
            Produto VARCHAR(100),
            Categoria_Produto VARCHAR(100),
            Preco FLOAT(10,2),
            Frete FLOAT(10,2),
            Data_Compra DATE,
            Vendedor VARCHAR(100),
            Local_Compra VARCHAR(100),
            Avaliacao_Compra INT,
            Tipo_Pagamento VARCHAR(100),
            Qntd_Parcelas INT,
            Latitude FLOAT(10,2),
            Longitude FLOAT(10,2),
            
            PRIMARY KEY (id));
    """)
except mysql.connector.Error as err:
    print("Failed creating table: {}".format(err))

**Selecionando a base de dados para verificar a tabela criada**

In [19]:
cursor.execute("USE dbprodutos;")
cursor.execute("SHOW TABLES;")

for tb in cursor:
  print(tb)

('tb_livros',)


## Inserindo os dados do csv na tabela

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do DataFrame para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

In [20]:
for i, row in df_livros.iterrows():
    print(tuple(row))

('65cce385ddadb93e4611d6c2', 'Modelagem preditiva', 'livros', 92.45, 5.6096965236, '2020-01-01', 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('65cce385ddadb93e4611d6fb', 'Modelagem preditiva', 'livros', 70.49, 1.337819504, '2023-02-01', 'Isabella Pereira', 'PE', 4, 'cupom', 1, -8.38, -37.86)
('65cce385ddadb93e4611d715', 'Ciência de dados com python', 'livros', 50.51, 10.2450858314, '2022-03-01', 'Thiago Silva', 'MG', 1, 'cartao_credito', 6, -18.1, -44.38)
('65cce385ddadb93e4611d719', 'Modelagem preditiva', 'livros', 71.64, 1.3990535443, '2023-03-01', 'Mariana Ferreira', 'SP', 4, 'cartao_credito', 2, -22.19, -48.79)
('65cce385ddadb93e4611d71b', 'Dashboards com Power BI', 'livros', 43.35, 3.2481049178, '2023-03-01', 'Rafael Costa', 'BA', 5, 'cartao_credito', 2, -13.29, -41.71)
('65cce385ddadb93e4611d72b', 'Iniciando em programação', 'livros', 83.18, 5.1160969115, '2021-04-01', 'Isabella Pereira', 'SP', 3, 'cartao_credito', 4, -22.19, -48.79)
('65cce385ddadb93e4611d740',

In [21]:
lista_dados = [tuple(row) for i, row in df_livros.iterrows()]
lista_dados

[('65cce385ddadb93e4611d6c2',
  'Modelagem preditiva',
  'livros',
  92.45,
  5.6096965236,
  '2020-01-01',
  'Thiago Silva',
  'BA',
  1,
  'cartao_credito',
  3,
  -13.29,
  -41.71),
 ('65cce385ddadb93e4611d6fb',
  'Modelagem preditiva',
  'livros',
  70.49,
  1.337819504,
  '2023-02-01',
  'Isabella Pereira',
  'PE',
  4,
  'cupom',
  1,
  -8.38,
  -37.86),
 ('65cce385ddadb93e4611d715',
  'Ciência de dados com python',
  'livros',
  50.51,
  10.2450858314,
  '2022-03-01',
  'Thiago Silva',
  'MG',
  1,
  'cartao_credito',
  6,
  -18.1,
  -44.38),
 ('65cce385ddadb93e4611d719',
  'Modelagem preditiva',
  'livros',
  71.64,
  1.3990535443,
  '2023-03-01',
  'Mariana Ferreira',
  'SP',
  4,
  'cartao_credito',
  2,
  -22.19,
  -48.79),
 ('65cce385ddadb93e4611d71b',
  'Dashboards com Power BI',
  'livros',
  43.35,
  3.2481049178,
  '2023-03-01',
  'Rafael Costa',
  'BA',
  5,
  'cartao_credito',
  2,
  -13.29,
  -41.71),
 ('65cce385ddadb93e4611d72b',
  'Iniciando em programação',
  'liv

In [22]:
sql = "INSERT INTO dbprodutos.tb_livros VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

cursor.executemany(sql, lista_dados)
cnx.commit()

In [23]:
print(cursor.rowcount, "dados foram inseridos.")

2226 dados foram inseridos.


In [24]:
df_2021_em_diante = pd.read_csv("../data/tb_2021_em_diante.csv")
df_2021_em_diante.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,65cce385ddadb93e4611d6cd,Mesa de centro,moveis,282.22,12.611805,2021-01-01,João Souza,SP,3,boleto,1,-22.19,-48.79
1,65cce385ddadb93e4611d6dc,Bicicleta,esporte e lazer,836.66,45.438043,2023-01-01,Larissa Alves,SP,5,cartao_credito,1,-22.19,-48.79
2,65cce385ddadb93e4611d6e7,Headset,eletronicos,340.22,15.700131,2021-02-01,Thiago Silva,SP,2,cupom,1,-22.19,-48.79
3,65cce385ddadb93e4611d6f0,Bicicleta,esporte e lazer,520.28,27.986471,2022-02-01,Rafael Costa,SP,5,cartao_credito,5,-22.19,-48.79
4,65cce385ddadb93e4611d6f7,Cômoda,moveis,417.02,19.789500,2023-02-01,Felipe Santos,PR,5,cartao_credito,2,-24.89,-51.55


In [27]:
try:
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS dbprodutos.tb_2021_em_diante(
            id VARCHAR(100),
            Produto VARCHAR(100),
            Categoria_Produto VARCHAR(100),
            Preco FLOAT(10,2),
            Frete FLOAT(10,2),
            Data_Compra DATE,
            Vendedor VARCHAR(100),
            Local_Compra VARCHAR(100),
            Avaliacao_Compra INT,
            Tipo_Pagamento VARCHAR(100),
            Qntd_Parcelas INT,
            Latitude FLOAT(10,2),
            Longitude FLOAT(10,2),
            
            PRIMARY KEY (id));
    """)
except mysql.connector.Error as err:
    print("Failed creating table: {}".format(err))

In [28]:
cursor.execute("USE dbprodutos;")
cursor.execute("SHOW TABLES;")

for tb in cursor:
  print(tb)

('tb_2021_em_diante',)
('tb_livros',)


In [29]:
lista_dados = [tuple(row) for i, row in df_2021_em_diante.iterrows()]
sql = "INSERT INTO dbprodutos.tb_2021_em_diante VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

cursor.executemany(sql, lista_dados)
cnx.commit()

In [30]:
print(cursor.rowcount, "dados foram inseridos.")

19722 dados foram inseridos.


## Visualizando os dados inseridos